<a href="https://colab.research.google.com/github/Sharekh1/Prediction-of-Product-Sales/blob/main/Prediction_of_Product_Sales.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Prediction of Product Sales

- Author: Mahmoud Abusharekh

## Project Overview


This project focuses on predicting sales of food items at various stores

## Load and Inspect Data


## Clean data

## Exploratory Data Analysis

## Feature Inspection